In [12]:
%matplotlib inline
%pylab inline
import numpy as np
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
matplotlib.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [13]:
TRAIN_DATA_PATH = "/Users/svegal/Downloads/train_{}/"
TRAIN_DATA_CHUNKS = (1, 2, 3)
PREICTAL = 1
INTERICTAL = 0

### Get the data

Load samples of interictal and preictal data

In [87]:
def mat_to_pandas(path):
    mat = loadmat(path, verify_compressed_data_integrity=False)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    return ndata['data']

In [88]:
import random

IGNORED_FILES = set(['.DS_Store'])

def is_class(class_id, filename):
    if any(list(map(lambda x: filename.endswith(x), IGNORED_FILES))):
        return False
    return int(filename.rstrip('.mat')[-1]) == class_id

def all_filepaths():
    result = []
    for i in TRAIN_DATA_CHUNKS:
        path = TRAIN_DATA_PATH.format(i)
        result.extend([path + f for f in os.listdir(path)])
    return result

def filenames_for_class(class_id, n_filenames):
    return random.sample([f for f in all_filepaths() if is_class(class_id, f)], n_filenames)

In [89]:
all_files = all_filepaths()
print(len(all_files))
print("Interictal", len([f for f in all_files if is_class(0, f)]))
print("Preictal", len([f for f in all_files if is_class(1, f)]))

6043
Interictal 5592
Preictal 450


In [428]:
from scipy.signal import resample
from scipy.fftpack import fft

def measures(d, axis):
    return [d.min(axis=axis), d.max(axis=axis), d.std(axis=axis), d.mean(axis=axis)]


def generate_features(data):
    data = data.transpose()
    N = 12000
    data = data.reshape(10, 24000, 16)
    to_stack = []
    time_data = resample(data, N, axis=1)
    freq_data = np.absolute(fft(time_data, axis=1))
    time_data = np.dstack([time_data, time_data.mean(axis=2)])
    freq_data = np.dstack([freq_data, freq_data.mean(axis=2)])
    for d in [time_data, freq_data]:
        to_stack.extend(measures(d, axis=1))
#         maxes, meanes, stds = [], [], []
#         for i in range(10):
#             cov = np.cov(d[i], rowvar=False)
#             maxes.append(cov.max())
#             meanes.append(cov.mean())
#             stds.append(cov.std())
#         to_stack.extend([maxes, meanes, stds])
    to_stack.append(freq_data.argmax(axis=1))
    return np.hstack(to_stack)

In [429]:
def train_dataset(n_interictal, n_preictal):
    """Create a train dataset from randomly chosen n_files, where the share of preictal data is share_preictal"""
    PATH = TRAIN_DATA_PATH.format(TRAIN_DATA_CHUNKS[0])
    X = None
    y = None
    files = None
    for i, n in enumerate([n_interictal, n_preictal]):
        filenames = filenames_for_class(i, n)
        for f in filenames:
            X_data = generate_features(mat_to_pandas(f))
            files_data = np.repeat(f, X_data.shape[0], axis=0).reshape((10, 1))
            y_data = np.repeat(i, X_data.shape[0], axis=0)
            X = np.vstack([X, X_data]) if X is not None else X_data
            y = np.concatenate([y, y_data]) if y is not None else y_data
            files = np.concatenate([files, files_data]) if files is not None else files_data
    return X, y


In [436]:
%time X_half, y_half = train_dataset(900, 450)
print(X_half.shape, y_half.shape)

CPU times: user 6min 31s, sys: 1min 43s, total: 8min 14s
Wall time: 8min 38s
(13500, 153) (13500,)


In [442]:
%time X, y = train_dataset(90, 45)
print(X.shape, y.shape)

CPU times: user 38.4 s, sys: 10.2 s, total: 48.6 s
Wall time: 49.9 s
(1350, 153) (1350,)


### Building the model

1. Divide 10 minute samples to non-overlapping 1 min samples
2. For each 1 minute sample calculate the probability
3. Use the mean over 10 samples to calculate the overall 10-minutes interval probability

In [437]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model

In [438]:
class Model(object):
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        self.clf = linear_model.LogisticRegression(penalty='l1', C=16, n_jobs=3, verbose=5)
        self.clf.fit(self.X_train, self.y_train)
        self.y_pred = self.clf.predict(self.X_test)

In [439]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

def metrics(model):
    funcs = dict(
        accuracy_score=accuracy_score, 
        precision_score=precision_score, 
        recall_score=recall_score,
        f1_score=f1_score,
        roc_auc_score=roc_auc_score,
    )
    result = {k: v(model.y_test, model.y_pred) for k, v in funcs.items()}
    for k, v in result.items():
        print(k.title(), v)
    return result

In [443]:
model_part = Model(X_half, y_half)

[LibLinear]

/usr/local/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [444]:
half_metrics = metrics(model_part)

F1_Score 0.195017793594
Precision_Score 0.472413793103
Roc_Auc_Score 0.527585420056
Recall_Score 0.122869955157
Accuracy_Score 0.664888888889


In [445]:
model2 = Model(X, y)
metrics = metrics(model2)

[LibLinear]F1_Score 0.19587628866
Precision_Score 0.246753246753
Roc_Auc_Score 0.44997486174
Recall_Score 0.162393162393
Accuracy_Score 0.538461538462


/usr/local/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Getting the results